In [1]:
import requests
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import date

In [39]:
team_urls = pd.read_csv('Spiele_Links.csv')
idx = 8
print(team_urls['team'][idx])
url = team_urls['url'][idx]
response = requests.get(url)
df = pd.DataFrame()

D2


In [40]:
#BeautifulSoup HTML-Dokument aus dem Quelltext parsen
soup= BeautifulSoup(response.text, 'html.parser')

In [41]:
#Tabelle aus dem HTML-Dokument extrahieren
table = soup.find('div', {"id":'id-team-matchplan-table'})
#print(table)

In [42]:
team=team_urls['team'][idx]
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)

for data in table.find_all('tbody'):
    dates = list()
    times = list()
    teams = list()
    rows = data.find_all('tr')
    kw = list()
    for i in range(1,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        date = rows[i].find('td', class_='column-date') #rows[i].text
        cleanString = re.sub(r'\ \|',',', date.text)
        cleanString = re.sub(r' ', '', cleanString)
        #removedSpace = re.sub(r'\t', '', removedSpace)
        splitted= cleanString.split(',')
        date_split = splitted[1].split('.')
        datum = datetime.date(int(date_split[2])+2000, int(date_split[1]), int(date_split[0]))
        kw.append(datum.isocalendar()[1])
        dates.append(splitted[1])
        times.append(splitted[2].encode('utf-8').decode('utf-8'))
        teams.append(team)
        
    df = pd.DataFrame(list(zip(dates,times,teams)),columns=['Datum','Zeit','Team'])
    df.insert(len(df.columns), 'KW', kw)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [6]:
if (df.shape[0]>0):
    #Club und Logo auslesen (befindet sich in jeder 3. Zeile ab 3. Zeile)
    for data in table.find_all('tbody'):
        rows = data.find_all('tr')
        club1 = list()
        club2 = list()
        for i in range(2,len(rows),3):
            #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
            #print(rows[i].text)
            removedSpace = re.sub(r'\n', '', rows[i].text)
            removedSpace = re.sub(r'\t', '', removedSpace)
            removedSpace = re.sub(r':', ';', removedSpace)
            splitted = removedSpace.split(';')
            club1.append(splitted[0])
            club2.append(splitted[1][0:-1])


        df.insert(len(df.columns), "Heim", club1)
        df.insert(len(df.columns), "Gast", club2)

In [7]:
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    logos_home = list()
    logos_guest= list()
    for i in range(2,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        logo = rows[i].find_all('div', class_="club-logo") #rows[i].text
        
        logo_link = re.findall(r'//(\S+)\">', str(logo))
        logos_home.append(logo_link[0])
        logos_guest.append(logo_link[1])
        
#found = re.search('AAA(.+?)ZZZ', string).group(1)

    df.insert(len(df.columns), 'Logo Heim', logos_home)
    df.insert(len(df.columns), 'Logo Gast', logos_guest)
    df['Heim'] = df['Heim'].replace(r'SGM.*ABI.*','SGM ABI', regex=True)
    df['Gast'] = df['Gast'].replace(r'SGM.*ABI.*','SGM ABI', regex=True)


In [8]:
df

,Datum,Zeit,Team,KW,Heim,Gast,Logo Heim,Logo Gast
0,19.07.23,19:30,A,29,SGM ABI,SG Sonnenhof Großaspach I,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
1,24.07.23,19:30,A,30,SGM ABI,FV Löchgau,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
2,02.09.23,16:00,A,35,Spfr. Untergriesheim,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
3,03.09.23,15:00,A,35,TuS Freiberg,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
4,09.09.23,16:30,A,36,SGM Markgröningen-Riexingen,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...


In [9]:
links = table.find_all('a')
print(len(links))
homeLinks = list();
guestLinks = list();
spielLinks = list();
for i in range(2, len(links),6):
    homeLinks.append(links[i].get('href'))
    guestLinks.append(links[i+1].get('href'))
    spielLinks.append(links[i+2].get('href'))
df.insert(len(df.columns), "home_link", homeLinks)
df.insert(len(df.columns), 'guest_link', guestLinks)
df.insert(len(df.columns), 'Spiel', spielLinks)
df

30


,Datum,Zeit,Team,KW,Heim,Gast,Logo Heim,Logo Gast,home_link,guest_link,Spiel
0,19.07.23,19:30,A,29,SGM ABI,SG Sonnenhof Großaspach I,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/mannschaft/sg-sonnenho...,https://www.fussball.de/spiel/sgm-sc-abstatt-b...
1,24.07.23,19:30,A,30,SGM ABI,FV Löchgau,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/mannschaft/fv-loechgau...,https://www.fussball.de/spiel/sgm-sc-abstatt-b...
2,02.09.23,16:00,A,35,Spfr. Untergriesheim,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/spfr-unterg...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/spiel/spfr-untergriesh...
3,03.09.23,15:00,A,35,TuS Freiberg,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/tus-freiber...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/spiel/tus-freiberg-sgm...
4,09.09.23,16:30,A,36,SGM Markgröningen-Riexingen,SGM ABI,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-markgro...,https://www.fussball.de/mannschaft/sgm-sc-abst...,https://www.fussball.de/spiel/sgm-markgroening...


In [10]:
#merge columns to table
if (idx==0):
    df.to_csv('out.csv', index=False, encoding="utf-8")
else:
    df.to_csv('out.csv', mode='a', index=False, header=False, encoding="utf-8")

